In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np
import tensorflow as tf
import cifar10
from cifar10 import num_classes
cifar10.maybe_download_and_extract()

Data has apparently already been downloaded and unpacked.


In [2]:
train_data,train_labels,onehot_train=cifar10.load_training_data()
test_data,test_labels,onehot_test=cifar10.load_test_data()
train_data=train_data.astype('float32')
train_labels=train_labels.astype('int32')
test_data=test_data.astype('float32')
test_labels=test_labels.astype('int32')
tf.logging.set_verbosity(tf.logging.ERROR)

Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_1
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_1'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_2
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_2'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_3
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_3'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_4
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_4'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_5
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_5'>
Loading data: data/CIFAR-10/cifar-10-batches-py/test_batch
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/test_batch'>


In [3]:
weight_decay = 1e-4
regularizer = tf.contrib.layers.l2_regularizer(scale=weight_decay)
filter_size = 3
feature_maps = 32

In [4]:
def cnn_model_fn(features, labels, mode):
      input_layer = tf.reshape(features["x"], [-1, 32, 32,3])
      conv1 = tf.contrib.layers.conv2d(
          inputs = input_layer,
          num_outputs = feature_maps,
          kernel_size = filter_size,
          weights_regularizer=regularizer,
          )
      
      conv1_bn =  tf.contrib.layers.batch_norm(
            inputs=conv1,
            fused = True
      )
        
      conv2 = tf.contrib.layers.conv2d(
          inputs = conv1_bn,
          num_outputs = feature_maps,
          kernel_size = filter_size,
          weights_regularizer=regularizer,
          )

      conv2_bn =  tf.contrib.layers.batch_norm(
            inputs=conv2,
            fused = True
        )
      

      pool1 =tf.contrib.layers.max_pool2d(inputs = conv2_bn,kernel_size = [2,2],stride = 2)
#       print ("********************************************\n")
#       print(pool1.shape)

      #dropout1 = tf.layers.dropout(inputs=pool1, rate=0.5, training=mode == tf.estimator.ModeKeys.TRAIN)
      dropout1 = tf.contrib.layers.dropout(inputs=pool1, keep_prob=0.8)

      conv3 = tf.contrib.layers.conv2d(
          inputs = dropout1,
          num_outputs = 2*feature_maps,
          kernel_size = filter_size,
          weights_regularizer=regularizer,
          )

      conv3_bn =  tf.contrib.layers.batch_norm(
            inputs=conv3,
            fused = True
        )

      conv4 = tf.contrib.layers.conv2d(
          inputs = conv3_bn,
          num_outputs = 2*feature_maps,
          kernel_size = filter_size,
          weights_regularizer=regularizer,
          )

      conv4_bn =  tf.contrib.layers.batch_norm(
            inputs=conv4,
            fused = True
        )

      
      pool2 = tf.contrib.layers.max_pool2d(inputs = conv4_bn,kernel_size = [2,2],stride = 2)
    
      dropout2 =  tf.contrib.layers.dropout(inputs=pool2, keep_prob=0.7)

      conv5 = tf.contrib.layers.conv2d(
          inputs = dropout2,
          num_outputs = 4*feature_maps,
          kernel_size = filter_size,
          weights_regularizer=regularizer,
          )


      conv5_bn =  tf.contrib.layers.batch_norm(
            inputs=conv5,
            fused = True
        )

      conv6 = tf.contrib.layers.conv2d(
          inputs = conv5_bn,
          num_outputs = 4*feature_maps,
          kernel_size = filter_size,
          weights_regularizer=regularizer,
          )

      conv6_bn =  tf.contrib.layers.batch_norm(
            inputs=conv6,
            fused = True
        )

      
      pool3 = tf.contrib.layers.max_pool2d(inputs = conv6_bn,kernel_size = [2,2] ,stride = 2)
      dropout3 =  tf.contrib.layers.dropout(inputs=pool3, keep_prob=0.6)
      

      pool3_flat = tf.reshape(dropout3, [-1, 4 * 4 * 128])
    
      logits = tf.layers.dense(inputs=pool3_flat, units=10)
      predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.contrib.layers.softmax(logits)
  }
      if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

      # Calculate Loss (for both TRAIN and EVAL modes)
      loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

      # Configure the Training Op (for TRAIN mode)
      if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.contrib.optimizer_v2.AdamOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

      # Add evaluation metrics (for EVAL mode)
      eval_metric_ops = {
          "accuracy": tf.metrics.accuracy(
              labels=labels, predictions=predictions["classes"]),
      }
      return tf.estimator.EstimatorSpec(
          mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [5]:
def current_loss():
    train_eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      shuffle=False)
    eval_result1=cifar_classifier.evaluate(input_fn=train_eval_input_fn)
    print("%%%%%%% Train accuracy %%%%%%%%%%%%\n",eval_result1)


    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": test_data},
      y=test_labels,
      num_epochs=1,
      shuffle=False)
    eval_result2=cifar_classifier.evaluate(input_fn=eval_input_fn)
    print("######### Test accuracy #############\n",eval_result2)

In [6]:
# Create the Estimator
cifar_classifier = tf.estimator.Estimator(
  model_fn=cnn_model_fn, model_dir="./cifar_model_teacher_adam_SOTA_ADAM2")


# Set up logging for predictions
# Log the values in the "Softmax" tensor with label "probabilities"
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=50)
print("started\n")
# Train the model
r = 140
for i in range (1,r):
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=64,
      num_epochs=None,
      shuffle=True)

    cifar_classifier.train(
      input_fn=train_input_fn,
      steps= 1000,
      hooks=None)
    # print("eva",type(eval_data))

    print ("\n\nEpoch = " )
    print(i)
    print("\n")
    current_loss()
print ("*****************END*******************")

started



Epoch = 
1


%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.96152002, 'loss': 0.11186291, 'global_step': 181000}
######### Test accuracy #############
 {'accuracy': 0.82179999, 'loss': 0.97985905, 'global_step': 181000}


Epoch = 
2


%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.96526003, 'loss': 0.10245406, 'global_step': 182000}
######### Test accuracy #############
 {'accuracy': 0.8168, 'loss': 0.929079, 'global_step': 182000}


Epoch = 
3




KeyboardInterrupt: 

In [ ]:
current_loss()